In [ ]:
import tensorflow as tf
import pandas as pd
import os
import datetime as dt
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
from prometheus_client import start_http_server, Gauge
import mlflow
import time
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from flask import Flask, request, jsonify
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize Flask app
app = Flask(__name__)

# Define Prometheus metrics
rmse_gauge = Gauge('model_rmse', 'Root Mean Square Error')
mae_gauge = Gauge('model_mae', 'Mean Absolute Error')
r2_gauge = Gauge('model_r2', 'R2 Score')
data_drift_gauge = Gauge('data_drift', 'Data Drift')
data_drift_score_gauge = Gauge('data_drift_score', 'Data Drift Score')
data_drift_percentage_gauge = Gauge('data_drift_porcentage', 'Data Drift Percentage')

def get_mlflow_metrics(run_id):
    client = mlflow.tracking.MlflowClient()
    metrics = client.get_run(run_id).data.metrics
    return metrics

def generate_evidently_metrics(reference_data, current_data):
    report = Report(metrics=[DataDriftPreset()])
    report.run(reference_data=reference_data, current_data=current_data)
    evidently_metrics = report.as_dict()
    #print(evidently_metrics)
    data_drift = evidently_metrics['metrics'][0]['result']['dataset_drift']
    data_drift_score = evidently_metrics['metrics'][1]['result']['drift_by_columns']['Open']['drift_score']
    drifted_features = evidently_metrics['metrics'][0]['result']['number_of_drifted_columns']
    total_features = evidently_metrics['metrics'][0]['result']['number_of_columns']
    data_drift_percentage = (drifted_features / total_features) * 100
    return data_drift_score, data_drift_percentage, data_drift

def load_model(run_id):
    model_uri = f"runs:/{run_id}/AMAZON_LSTM"
    model = mlflow.keras.load_model(model_uri)
    return model

def load_and_preprocess_data(file_path, start_date, end_date):
    df = pd.read_csv(file_path)
    df = df[['Date', 'Open']]
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    
    train_data = df.loc[dt.datetime(2000, 4, 1):dt.datetime(2003, 4, 1)]
    
    data = df.loc[start_date:end_date]
    
    test_data = data
    
    scaler = MinMaxScaler(feature_range=(0,1))

    # Selecting Open values
    dataset_test = test_data.Open.values 
    # Reshaping 1D to 2D array
    dataset_test = np.reshape(dataset_test, (-1,1)) 
    # Normalizing values between 0 and 1
    scaled_test = scaler.fit_transform(dataset_test) 

    # Preparing test data for LSTM
    test = []
    for i in range(50, len(scaled_test)):
        test.append(scaled_test[i-50:i, 0])

    # The data is converted to numpy array
    test = np.array(test)
    # Reshaping
    test = np.reshape(test, (test.shape[0], test.shape[1],1))

    y_test = scaled_test[50:]
    y_test = scaler.inverse_transform(y_test)

    return train_data, test_data, test, y_test, scaler

def make_predictions(model, test_data, scaler):
    predictions = model.predict(test_data)
    predictions = scaler.inverse_transform(predictions)
    return predictions

def log_metrics_to_mlflow(rmse, mae, r2, data_drift_score, data_drift_percentage, data_drift, run_id):
    # Log metrics to the existing run
    with mlflow.start_run(run_id=run_id):
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("data_drift_score", data_drift_score)
        mlflow.log_metric("data_drift", data_drift)
        mlflow.log_metric("data_drift_percentage", data_drift_percentage)

def update_metrics(run_id, reference_data, current_data):
    metrics = get_mlflow_metrics(run_id)
    rmse_gauge.set(metrics.get('rmse', 0))
    mae_gauge.set(metrics.get('mae', 0))
    r2_gauge.set(metrics.get('r2', 0))

    data_drift_score, data_drift_percentage, data_drift  = generate_evidently_metrics(reference_data, current_data)
    data_drift_score_gauge.set(data_drift_score)
    data_drift_percentage_gauge.set(data_drift_percentage)
    data_drift_gauge.set(data_drift)
    return data_drift_score, data_drift_percentage, data_drift

def calculate_and_update_metrics(model, test_data, y_true, scaler, run_id, reference_data, current_data):
    y_pred = make_predictions(model, test_data, scaler)
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    rmse_gauge.set(rmse)

    # Calculate MAE
    mae = mean_absolute_error(y_true, y_pred)
    mae_gauge.set(mae)

    # Calculate R2 Score
    r2 = r2_score(y_true, y_pred)
    r2_gauge.set(r2)

    # Update data drift metrics
    data_drift_score, data_drift_percentage, data_drift = update_metrics(run_id, reference_data, current_data)

    # Log all metrics to MLflow
    log_metrics_to_mlflow(rmse, mae, r2, data_drift_score, data_drift_percentage, data_drift, run_id)

    # Logging the calculated metrics
    print(f"Calculated Metrics - RMSE: {rmse}, MAE: {mae}, R2: {r2}, Data Drift Score: {data_drift_score}, Data Drift Percentage: {data_drift_percentage}%")
    
    # Create a DataFrame with the metrics
    metrics_df = pd.DataFrame({
        'RMSE': [rmse],
        'MAE': [mae],
        'R2': [r2],
        'Data Drift Score': [data_drift_score],
        'Data Drift Percentage': [data_drift_percentage]
    })

    # Append the metrics to the CSV file
    try:
        existing_df = pd.read_csv("ayuda.csv")
        combined_df = pd.concat([existing_df, metrics_df], ignore_index=True)
    except FileNotFoundError:
        combined_df = metrics_df

    combined_df.to_csv("ayuda.csv", index=False)
    
    return y_pred

@app.route('/predict', methods=['POST'])
def trigger_prediction():
    run_id = request.json.get('run_id')
    file_path = request.json.get('file_path')
    start_date = request.json.get('start_date')
    end_date = request.json.get('end_date')
    
    if not all([run_id, file_path, start_date, end_date]):
        return jsonify({"error": "run_id, file_path, start_date, and end_date are required"}), 400

    start_date = dt.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = dt.datetime.strptime(end_date, '%Y-%m-%d')

    train_data, test_data, test, y_test, scaler = load_and_preprocess_data(file_path, start_date, end_date)
    
    model = load_model(run_id)
    y_pred = calculate_and_update_metrics(model, test, y_test, scaler, run_id, train_data, test_data)
    
    global current_train_data, current_test_data, current_test, current_y_test, current_model, current_scaler
    current_train_data = train_data
    current_test_data = test_data
    current_test = test
    current_y_test = y_test
    current_model = model
    current_scaler = scaler

    return jsonify({"message": "Prediction and metrics update successful"})

if __name__ == '__main__':
    # Initial values
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    run_id = '6852c887552646bcbe3f8c8e1aa98035'  # Replace with your MLflow run ID
    file_path = 'directory/amzn.us.txt'
    start_date = dt.datetime(2010, 4, 1)
    end_date = dt.datetime(2017, 4, 1)

    train_data, test_data, test, y_test, scaler = load_and_preprocess_data(file_path, start_date, end_date)
    model = load_model(run_id)
    
    # Setting global variables to be used in the main loop
    current_train_data = train_data
    current_test_data = test_data
    current_test = test
    current_y_test = y_test
    current_model = model
    current_scaler = scaler

    start_http_server(8000)  # Prometheus will scrape metrics from http://localhost:8000
    
    # Start Flask server
    app.run(port=5001)  # The Flask server will run on http://localhost:5001

    while True:
        y_pred = calculate_and_update_metrics(current_model, current_test, current_y_test, current_scaler, run_id, current_train_data, current_test_data)
        time.sleep(10)  # Update metrics every 10 seconds




2024-06-03 19:33:05.331003: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 19:33:06.051389: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 * Serving Flask app '__main__'
 * Debug mode: off


2024-06-03 19:33:08.831879: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-03 19:33:08.857106: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
 * Running on http://127.0.0.1:5001
Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step


127.0.0.1 - - [03/Jun/2024 19:33:14] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jun/2024 19:33:14] "POST /predict HTTP/1.1" 200 -


...

127.0.0.1 - - [03/Jun/2024 20:41:54] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jun/2024 20:41:54] "POST /predict HTTP/1.1" 200 -


Calculated Metrics - RMSE: 17.977162530917617, MAE: 13.682617938701933, R2: -0.48844250315472015, Data Drift Score: 2.5336454827554905e-95, Data Drift Percentage: 100.0%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step


127.0.0.1 - - [03/Jun/2024 20:42:55] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jun/2024 20:42:55] "POST /predict HTTP/1.1" 200 -


Calculated Metrics - RMSE: 8.159124759733027, MAE: 6.678674692007226, R2: 0.779777709204304, Data Drift Score: 2.5336454827554905e-95, Data Drift Percentage: 100.0%


ERROR:__main__:Exception on /predict [POST]
Traceback (most recent call last):
  File "/home/mguzman/venv/lib/python3.10/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/mguzman/venv/lib/python3.10/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/mguzman/venv/lib/python3.10/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/mguzman/venv/lib/python3.10/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "/tmp/ipykernel_2998/864047430.py", line 169, in trigger_prediction
    train_data, test_data, test, y_test, scaler = load_and_preprocess_data(file_path, start_date, end_date)
  File "/tmp/ipykernel_2998/864047430.py", line 78, in load_and_preprocess_data
    test = np.reshape(test,

In [7]:
e = {'metrics': [{'metric': 'DatasetDriftMetric', 'result': {'drift_share': 0.5, 'number_of_columns': 1, 'number_of_drifted_columns': 1, 'share_of_drifted_columns': 1.0, 'dataset_drift': True}}, {'metric': 'DataDriftTable', 'result': {'number_of_columns': 1, 'number_of_drifted_columns': 1, 'share_of_drifted_columns': 1.0, 'dataset_drift': True, 'drift_by_columns': {'Open': {'column_name': 'Open', 'column_type': 'num', 'stattest_name': 'K-S p_value', 'stattest_threshold': 0.05, 'drift_score': 2.7995936562559147e-125, 'drift_detected': True, 'current': {'small_distribution': {'x': [730.0, 744.7, 759.4, 774.1, 788.8, 803.5, 818.2, 832.9, 847.6, 862.3, 877.0], 'y': [0.0018020453214398285, 0.004054601973239646, 0.015317385232238544, 0.008109203946479291, 0.0045051133035995715, 0.005856647294679444, 0.007208181285759369, 0.01126278325899893, 0.009010226607199214, 0.0009010226607199142]}}, 'reference': {'small_distribution': {'x': [284.4, 333.15, 381.9, 430.65, 479.4, 528.15, 576.9, 625.65, 674.4, 723.15, 771.9], 'y': [0.005652872979605652, 0.002877210798002877, 0.0014893797072014893, 0.001117034780401117, 0.0012862824744012862, 0.0022002200220022, 0.0014555301684014556, 0.0014893797072014893, 0.001590928323601591, 0.001353981552001354]}}}}, 'current_fi': None, 'reference_fi': None}}]}
print(e)

{'metrics': [{'metric': 'DatasetDriftMetric', 'result': {'drift_share': 0.5, 'number_of_columns': 1, 'number_of_drifted_columns': 1, 'share_of_drifted_columns': 1.0, 'dataset_drift': True}}, {'metric': 'DataDriftTable', 'result': {'number_of_columns': 1, 'number_of_drifted_columns': 1, 'share_of_drifted_columns': 1.0, 'dataset_drift': True, 'drift_by_columns': {'Open': {'column_name': 'Open', 'column_type': 'num', 'stattest_name': 'K-S p_value', 'stattest_threshold': 0.05, 'drift_score': 2.7995936562559147e-125, 'drift_detected': True, 'current': {'small_distribution': {'x': [730.0, 744.7, 759.4, 774.1, 788.8, 803.5, 818.2, 832.9, 847.6, 862.3, 877.0], 'y': [0.0018020453214398285, 0.004054601973239646, 0.015317385232238544, 0.008109203946479291, 0.0045051133035995715, 0.005856647294679444, 0.007208181285759369, 0.01126278325899893, 0.009010226607199214, 0.0009010226607199142]}}, 'reference': {'small_distribution': {'x': [284.4, 333.15, 381.9, 430.65, 479.4, 528.15, 576.9, 625.65, 674.4

In [4]:
e['metrics'][0]['result']['dataset_drift']

True

In [14]:
e['metrics'][1]['result']['drift_by_columns']['Open']['drift_score']

2.7995936562559147e-125